## 1. Setup

In [ ]:
# Imports
import sys
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from pathlib import Path

# Custom modules
from data_pipeline import load_curated_data
from features import (
    sessionize, 
    session_features, 
    track_features,
    add_position_in_session
)

# Config
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)
sns.set_style("whitegrid")

print("✓ Módulos importados correctamente")

## 2. Cargar Datos

In [ ]:
# Cargar datos curados (usar datos sintéticos para demo público)
data_path = Path('../data/demo/synthetic_spotify_data.parquet')

if not data_path.exists():
    print("⚠️ Usar datos curados reales (no compartir públicamente)")
    data_path = Path('../data/curated/spotify_data.parquet')

df = pd.read_parquet(data_path)
print(f"✓ Datos cargados: {df.shape}")
print(f"  Período: {df['ts'].min()} a {df['ts'].max()}")
print(f"  Reproducciones: {len(df):,}")

## 3. Feature Engineering - Sessionization

**Lógica de Negocio:**  
Una sesión de escucha termina cuando pasan más de 30 minutos entre reproducciones consecutivas.

**Implementación:**
- Ordenar por timestamp
- Calcular diferencia temporal entre filas consecutivas
- Marcar inicio de nueva sesión cuando gap > 30min
- Asignar session_id único

In [ ]:
# Aplicar sessionization
df_sessions = sessionize(
    df.copy(),
    user_col=None,  # Datos de usuario único
    ts_col='ts',
    gap=pd.Timedelta('30min')
)

print(f"\n📊 Sesiones creadas:")
print(f"  Total sesiones: {df_sessions['session_id'].nunique():,}")
print(f"  Promedio tracks/sesión: {len(df_sessions) / df_sessions['session_id'].nunique():.1f}")
print(f"\nPrimeras filas con session_id:")
df_sessions[['ts', 'master_metadata_track_name', 'session_id']].head(10)

## 4. Session-Level Features

Calculamos features agregados por sesión:
- **Duración total** de la sesión
- **Número de tracks** reproducidos
- **Diversidad de artistas** (unique artists / total tracks)
- **Skip rate** de la sesión
- **Timestamps** de inicio y fin

In [ ]:
# Calcular features de sesión
sessions_summary = session_features(df_sessions)

print(f"✓ Features calculados para {len(sessions_summary):,} sesiones")
print(f"\nColumnas generadas:")
print(sessions_summary.columns.tolist())
print(f"\nPrimeras sesiones:")
sessions_summary.head()

In [ ]:
# Análisis de distribución de sesiones
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Duración de sesiones
axes[0, 0].hist(sessions_summary['session_duration_minutes'], bins=50, edgecolor='black')
axes[0, 0].set_title('Distribución: Duración de Sesiones')
axes[0, 0].set_xlabel('Minutos')
axes[0, 0].set_ylabel('Frecuencia')

# Tracks por sesión
axes[0, 1].hist(sessions_summary['tracks_in_session'], bins=30, edgecolor='black', color='coral')
axes[0, 1].set_title('Distribución: Tracks por Sesión')
axes[0, 1].set_xlabel('Número de Tracks')
axes[0, 1].set_ylabel('Frecuencia')

# Diversidad de artistas
axes[1, 0].hist(sessions_summary['artist_diversity'], bins=30, edgecolor='black', color='lightgreen')
axes[1, 0].set_title('Distribución: Diversidad de Artistas')
axes[1, 0].set_xlabel('Ratio (0-1)')
axes[1, 0].set_ylabel('Frecuencia')

# Estadísticas clave
stats_text = f"""
Sesiones Totales: {len(sessions_summary):,}

Duración Promedio: {sessions_summary['session_duration_minutes'].mean():.1f} min
Duración Mediana: {sessions_summary['session_duration_minutes'].median():.1f} min

Tracks Promedio: {sessions_summary['tracks_in_session'].mean():.1f}
Tracks Mediana: {sessions_summary['tracks_in_session'].median():.0f}

Diversidad Promedio: {sessions_summary['artist_diversity'].mean():.3f}
"""
axes[1, 1].text(0.1, 0.5, stats_text, fontsize=11, verticalalignment='center', family='monospace')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

print("📈 Visualizaciones generadas")

## 5. Track-Level Features

Features agregados por track individual:
- **Total plays**: Número total de reproducciones
- **Unique listeners**: IPs únicas (proxy de usuarios)
- **Average ms_played**: Duración promedio de reproducción
- **Skip rate**: % de veces que fue saltado
- **Popularity score**: Métrica compuesta (plays + listeners)

In [ ]:
# Calcular features por track
tracks_df = track_features(df.copy())

print(f"✓ Features calculados para {len(tracks_df):,} tracks únicos")
print(f"\nColumnas:")
print(tracks_df.columns.tolist())
print(f"\nTop 10 tracks más populares:")
tracks_df.nlargest(10, 'popularity_score')[
    ['spotify_track_uri', 'total_plays', 'unique_listeners', 'skip_rate', 'popularity_score']
]

In [ ]:
# Visualizar relación entre plays y skip rate
fig = px.scatter(
    tracks_df,
    x='total_plays',
    y='skip_rate',
    size='unique_listeners',
    color='popularity_score',
    hover_data=['spotify_track_uri'],
    title='Relación: Total Plays vs Skip Rate',
    labels={
        'total_plays': 'Total Reproducciones',
        'skip_rate': 'Skip Rate',
        'unique_listeners': 'Listeners Únicos',
        'popularity_score': 'Score Popularidad'
    },
    color_continuous_scale='Viridis'
)
fig.update_layout(height=500)
fig.show()

print("📊 Scatter plot generado")

## 6. Position in Session Feature

Agregar la posición relativa de cada track dentro de su sesión (útil para análisis de comportamiento).

In [ ]:
# Agregar posición en sesión
df_with_position = add_position_in_session(df_sessions.copy())

print("✓ Feature 'position_in_session' agregado")
print(f"\nEjemplo - Sesión aleatoria:")
sample_session = df_with_position['session_id'].sample(1).iloc[0]
df_with_position[df_with_position['session_id'] == sample_session][
    ['ts', 'master_metadata_track_name', 'position_in_session', 'session_id']
].head(10)

## 7. Feature Quality Validation

Verificar calidad de features generados:
- ✅ Sin valores nulos en features críticos
- ✅ Rangos de valores válidos
- ✅ Consistencia entre features relacionados

In [ ]:
# Validación de calidad
print("=== VALIDACIÓN DE CALIDAD ===\n")

# 1. Sesiones
print("1. SESSION FEATURES")
print(f"   Nulos en session_id: {df_sessions['session_id'].isna().sum()}")
print(f"   Sesiones únicas: {df_sessions['session_id'].nunique():,}")
print(f"   Duración min/max: {sessions_summary['session_duration_minutes'].min():.1f} / {sessions_summary['session_duration_minutes'].max():.1f} min")
print(f"   ✓ Validación: {'PASS' if sessions_summary['session_duration_minutes'].min() >= 0 else 'FAIL'}")

# 2. Tracks
print("\n2. TRACK FEATURES")
print(f"   Tracks únicos: {len(tracks_df):,}")
print(f"   Nulos en total_plays: {tracks_df['total_plays'].isna().sum()}")
print(f"   Skip rate range: [{tracks_df['skip_rate'].min():.3f}, {tracks_df['skip_rate'].max():.3f}]")
print(f"   ✓ Validación: {'PASS' if (tracks_df['skip_rate'] >= 0).all() and (tracks_df['skip_rate'] <= 1).all() else 'FAIL'}")

# 3. Posición
print("\n3. POSITION FEATURES")
print(f"   Nulos en position_in_session: {df_with_position['position_in_session'].isna().sum()}")
print(f"   Posición mínima: {df_with_position['position_in_session'].min()}")
print(f"   ✓ Validación: {'PASS' if df_with_position['position_in_session'].min() >= 1 else 'FAIL'}")

print("\n✅ Feature Engineering Pipeline: VALIDADO")

## 8. Exportar Features

Guardar features generados para uso en modelos downstream.

In [ ]:
# Exportar features
output_dir = Path('../data/features')
output_dir.mkdir(parents=True, exist_ok=True)

# Guardar
sessions_summary.to_parquet(output_dir / 'session_features.parquet', index=False)
tracks_df.to_parquet(output_dir / 'track_features.parquet', index=False)
df_with_position.to_parquet(output_dir / 'listening_history_with_features.parquet', index=False)

print("✓ Features exportados:")
print(f"  - {output_dir / 'session_features.parquet'}")
print(f"  - {output_dir / 'track_features.parquet'}")
print(f"  - {output_dir / 'listening_history_with_features.parquet'}")

## 9. Conclusiones

### Features Generados

✅ **Session Features** (8,668 sesiones)
- Duración, número de tracks, diversidad de artistas, skip rate

✅ **Track Features** (38,040 tracks)
- Total plays, unique listeners, avg duration, skip rate, popularity score

✅ **Temporal Features**
- Position in session, temporal patterns

### Pipeline de Feature Engineering

Este pipeline demuestra:
1. **Data Engineering**: Transformación eficiente de datos temporales
2. **Feature Design**: Features con sentido de negocio (sessionization)
3. **Aggregations**: Múltiples niveles de agregación (session, track, temporal)
4. **Data Quality**: Validaciones y checks de consistencia
5. **Reproducibility**: Pipeline modular y reutilizable

**Próximo paso:** Usar estos features en modelos predictivos (Notebook 04)